---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

medical_dates_df = pd.Series(doc)

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data.

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates.

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import itertools
import random
import re
from functools import lru_cache
import numpy as np
import pandas as pd
import typing
import datetime

In [3]:
class NicDate(dict):
    """
    The NicDate Class inputs the string values from regex search for month, day and year.
    It inherits from dictionary.
    A dataclass is used instead of a raw dictionary, because a dictionary is not hashable, and can not be cached by
    @lru_cache. This class is  hashable and able to be reused in @lru_cache functions.

    :raises: Value Error if verify_not_none method is called and
    """

    def __hash__(self):
        # Define a specific has for this class, so it can be cached.
        hash_seed = hash(f"{self.month}/{self.date}/{self.year}")
        return hash_seed

    def __init__(self, month: str = str(), date: str = str(), year: str = ()):
        super().__init__()
        self.month = month
        self.date = date
        self.year = year

    def __repr__(self):
        return f"Day = {self.date} Month = {self.month} Year = {self.year}"

    @lru_cache(maxsize=2)
    @staticmethod
    def month_names_short() -> tuple:
        mns = ('jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec')
        return mns

    @lru_cache(maxsize=2)
    @staticmethod
    def month_names_long() -> tuple:
        mnl = ('january', 'february', 'march', 'april','May','june', 'july', 'august', 'september', 'october', 'november',
               'december',)
        return mnl

    def verify_not_none(self):
        """
        This function returns an error when called if any of the components are none.

        :raises ValueError if any of the components are None.
        """
        if any((self.month is None, self.date is None, self.year is None)):
            raise ValueError("None Value passed")

    @staticmethod
    @lru_cache(maxsize=12)
    def month_conversion_dict() -> dict:
        """
        The purpose of this function is to create a dictionary where keys are text which can represent a month, and the
        numbers are the sequence of the month from 1-12

        :return: dictionary where the keys are the short names of the month, and the values are the numerical sequence of the month.
        """
        # Array is written as numbers 1-12 rather than generated because previous testing has shown direct read to be
        # faster
        month_numbers = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)
        month_conversion_dict = {a: b for a, b in zip(NicDate.month_names_short(), month_numbers)}
        month_conversion_dict.update({a: b for a, b in zip(NicDate.month_names_long(), month_numbers)})
        return month_conversion_dict

    @staticmethod
    def find_largest_day(result_month: int = 0) -> dict:
        """
        The purpose of this function is to determine the largest day in the month. It supports date validation utilities
        At some point I'd like to add leap year functionality to it
        TODO: Add leap year functionality someday.
        :param result_month: integer
        :return: integer
        """
        largest_day_dict = {1: 31, 2: 29, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
        largest_day = largest_day_dict.get(result_month, np.nan)
        return largest_day

    def is_valid_year(self, year: typing.Union[int, str] = None) -> bool:
        """
        The purpose of this function is to determine whether a year (either int or string) is a valid year.
        Accommodates two digits and four digit values.

        :param year: Either string or integer representing year
        :return: bool True if  year is valid.
        """
        valid_year = True
        # ~~~~~~~~~~~~~~~~~~~
        # Test the year first
        # ~~~~~~~~~~~~~~~~~~~
        if year is None:
            valid_year = False
            return valid_year

        # ~~~~~~~~~~~~~~~~~~~
        # Accommodate Strings
        # ~~~~~~~~~~~~~~~~~~~
        if not isinstance(year, int):
            year = int(year)

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Check between 0 and 100 for two digit range (pretty much any two digit)
        # Check between 1900 and 2030 for a valid year digit.
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Note- Any is used here to speed up analysis with switching.
        if not any((0 < year < 100, 1900 < year < 2030)):
            valid_year = False
        return valid_year

    def is_valid_month(self, month: typing.Union[int, str] = None):
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # None is an acceptable value for month.
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        if month is None:
            return True

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Handle the case where digits are passed either directly or as a non string
        #  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        if (isinstance(month, str) and month.isdigit()) or isinstance(month, int):
            valid_month = 0 < int(month) < 13
            return valid_month

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Handle the name case where a name is passed.
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        elif isinstance(month, str):
            valid_month = any((month.lower() in self.month_names_short, month.lower() in self.month_names_long))
            return valid_month

    def is_valid_day(self, month: typing.Union[int, str] = None, day: typing.Union[int, str] = None):

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # If there's no month, then the day doesn't matter, so its ok
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        if month is None:
            return True

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # By clearing non valid months here, we eliminate having to put a lot of error checking in the code below.
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        if not self.is_valid_month(month):
            return False

        if month.isalpha():
            month = self.month_conversion_dict().get(month, None)
        else:
            month = int(month)
        day = np.uint8(day)
        is_valid_day = 0 < day < self.find_largest_day(result_month=month)
        return is_valid_day

    def valid_date(self, in_match_result: re.Match):
        """
        The purpose of this function is to determine the validity of a match object returned from regex.
        It returns a score from 0 to 8 with 8 being the most likely valid date, and 0 being not a valid date, and 8
        being the most likely date.
        :param in_match_result: A dictionary generated by group date of regex.
        :return: integer from 0-8
        :rtype np.uint8:
        """

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # First test the year. If the year isn't valid then forget about it
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        in_match_result = in_match_result.groupdict()

        year_result = self.is_valid_year(year=in_match_result.get('year', None))
        if not year_result:
            return np.uint8(0)
        else:
            validity_score = 1

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Test the month. a Null Month is OK, else bump validity score to 4
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        month_result = self.is_valid_month(month=in_match_result.get('month', None))
        if not month_result:
            return validity_score
        else:
            validity_score += 3

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Test the day.
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        day_result = self.is_valid_day(month=in_match_result.get('month', None), day=in_match_result.get('day', None))
        if day_result:
            validity_score += 4
        return validity_score

In [4]:

class DateFinderx:
    """
    The purpose of this class is to find dates in text strings.
    :param raw_text: Raw text from which a date is to be extracted.
    :
    """

    def __repr__(self):
        """
        :return: Date in american mm/dd/yyyy format (all numeric)
        """
        return f"{self.month}/{self.day}/{self.year}"

    def __init__(self, raw_text: str = ''):
        """


        """
        self.month = None
        self.day = None
        self.year = None

        cleaned_text = self.clean_text(raw_text=raw_text)
        result_dict = self.apply_regexes(search_text=cleaned_text)
        self.pydate = self.create_pydate(result_dict=result_dict)

    def clean_text(self, raw_text: str = None) -> str:
        """
        The purpose of this function is to aid regex effectiveness by cleaning out character (period, comma, colon,
        semicolon) from the test string. Double spaces are also eliminated.

        :param raw_text: A string which will be used for date analysis.
        """
        clean_text = re.sub(pattern=r"[.,;:]", string=raw_text, repl='')
        cleaner_text = re.sub(pattern=r"\s{2,}", string=clean_text, repl=' ')
        return cleaner_text

    def apply_regexes(self, search_text: str = str()):
        """
        This function applies the regex patterns to the string, iterating until a match is found.
        :param search_text: String to search for a date with regexes It outputs a group dictionary
        :return: dictionary of month, day and year
        """
        date_capture_regex = DateCaptureRegex.create_date_regex()

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Iterate through the regexes. Have three cases, first, no match, just continue iterating
        # second case, one result, grab the group dictionary of the first.
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        for dcr in date_capture_regex:
            results = tuple(re.finditer(pattern=dcr, string=search_text))
            if len(results) == 0:
                continue
            if len(results) == 1:
                return results[0].groupdict()
            if len(results) > 1:
                validity_results = [(result, NicDate().valid_date(in_match_result=result)) for result in results]
                validity_results = sorted(validity_results, key=lambda x: x[1], reverse=True)
                best_result = validity_results[0][0].groupdict()
                return best_result

    def create_pydate(self, result_dict: dict = None):
        """
        The purpose of this function is to take the results from regex, delivered in a dictionary format and
        produce a python date object.
        :param result_dict: group dictionary of the date regex result.
        :return: Python Date object representing the values of the dictionary.
        """

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # In the first section here, we get the year result. The year can be wither two or four digits.
        # We extract the year as an integer. Two digit will yield a result less than 100. In which case
        # (According to the requirements of the project we assume all years are 19XX) we just add 1900.
        # Otherwise, we pass along the year as an integer.
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        year_seed = int(result_dict.get('year'))
        if year_seed < 100:
            year_seed = year_seed + 1900
        self.year = year_seed

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Now we extract the month. According to the specs, if the month is not present (None in Regex)
        # Then we assume it's the first month of the year. Month can come in as a name "June" or a number
        # in string format form the regex.  If it's a name, we use the month conversion dictionary
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        month_seed = result_dict.get('month', '1').lower()
        if month_seed.isalpha():
            month_num = int(NicDate.month_conversion_dict().get(month_seed, None))
        else:
            month_num = int(month_seed)
            if not NicDate().is_valid_month(month=month_num):
                month_num = 1

        self.month = month_num

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        #  Extract the day. Day should always come in as numeric; so no high processing needed.
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        self.day = int(result_dict.get('day', 1))

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        #  create a datetime.date object with the info.
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        try:
            return datetime.date(year=self.year, month=self.month, day=self.day)
        except ValueError as uhoh:
            print(f"{uhoh}")


In [5]:


class DateCaptureRegex:
    """
    This is a wrapper class for the create_date_regex function.
    It has been separated into a class for future scalability of regex.
    """

    def __init__(self):
        pass

    def __hash__(self):
        """
        Hash required for using lru_cache.
        :return: int
        """

        return random.getrandbits(63)

    @lru_cache(maxsize=2)
    @staticmethod
    def create_date_regex() -> tuple:
        from text_mining_package import NicDate
        """
        This function creates a tuple of regex patterns to be used in searching for date.
        :return: re.Pattern - regex pattern to find dates.
        """

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Create list of month names for iteration
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        months = tuple(itertools.chain(NicDate.month_names_short(), NicDate.month_names_long()))

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Create the initial regex strings which will catch numeric instances of dates and 3 letter months
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        regex_builder = [r"(?P<a>(?P<month>\d{1,2})/(?P<day>\d{1,2})/(?P<year>\d{4}))",
                         r"(?P<b>(?P<month>\d{1,2})/(?P<day>\d{1,2})/(?P<year>\d{2}))",
                         r"(?P<c>(?P<month>\d{1,2})\-(?P<day>\d{1,2})\-(?P<year>\d{4}))",
                         r"(?P<d>(?P<month>\d{1,2})\-(?P<day>\d{1,2})\-(?P<year>\d{2}))",
                         r"(?P<e>(?P<day>\d{1,2})\s+(?P<month>[a-zA-Z]{3})\s+(?P<year>\d{4}))"]

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Build strings with different patterns, but for each month
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        regex_builder.extend([fr"(?P<f>(?P<day>\d\d)\s+(?P<month>{x})\s+(?P<year>\d\d\d\d))" for x in months])
        regex_builder.extend([fr"(?P<g>(?P<day>\d)\s+(?P<month>{x})\s+(?P<year>\d\d\d\d))" for x in months])
        regex_builder.extend([fr"(?P<h>(?P<day>\d)\s+(?P<month>{x})\s+(?P<year>\d\d))" for x in months])
        regex_builder.extend([fr"(?P<i>(?P<day>\d\d)\s+(?P<month>{x})\s+(?P<year>\d\d))" for x in months])
        regex_builder.extend([fr"(?P<j>(?P<month>{x})\s+(?P<day>\d\d)\s+(?P<year>\d\d\d\d))" for x in months])
        regex_builder.extend([fr"(?P<k>(?P<month>{x})\s+(?P<day>\d)\s+(?P<year>\d\d\d\d))" for x in months])
        regex_builder.extend([fr"(?P<m>(?P<month>{x})\s+(?P<day>\d\d)\s+(?P<year>\d\d))" for x in months])
        regex_builder.extend([fr"(?P<l>(?P<month>{x})\s+(?P<day>\d)\s+(?P<year>\d\d))" for x in months])
        regex_builder.extend([fr"(?P<o>(?P<month>{x})\s+(?P<year>\d\d\d\d))" for x in months])
        regex_builder.extend([fr"(?P<n>(?P<month>{x})\s+(?P<year>\d\d))" for x in months])
        regex_builder.append(r"(?P<p>(?P<month>\d{1,2})[\s+/](?P<year>\d{4}))")
        regex_builder.append(r"(?P<q>(?P<month>\d{1,2})[\s+/](?P<year>\d{2}))")
        regex_builder.extend([fr"(?P<year>{x})" for x in range(1900, 2050)])
        regex_tuple = tuple([re.compile(pattern=regex_build, flags=re.IGNORECASE) for regex_build in regex_builder])
        return regex_tuple


In [10]:
import copy

def date_sorter():
    
     # ~~~~~~~~~~~~~~~~~~~~~~~~
    # read in the source file
    # ~~~~~~~~~~~~~~~~~~~~~~~~
    doc = []
    with open(r'.\dates.txt') as file:
        for line in file:
            doc.append(line)

    original_date_series = pd.Series(data= copy.copy(doc))
    extraction_series_a = pd.Series(data=original_date_series\
                                    .apply(lambda x: DateFinderx(raw_text=x).pydate))
    series_as_pandas_dt = pd.to_datetime(extraction_series_a)\
        .dt\
        .date\
        .sort_values()\
        .index\
        .to_series()\
        .reset_index(drop=True)

    return series_as_pandas_dt


In [11]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64